In [ ]:
import os
import pandas as pd

# Load from project root /data folder
project_root = os.path.dirname(os.getcwd())  # Go up from notebooks/
data_path = os.path.join(project_root, 'data', 'simulated_transactions.csv')

print(f"Loading from: {data_path}")

df_full = pd.read_csv(data_path)
print(f"Dataset loaded: {df_full.shape}")
print("Columns:", list(df_full.columns[:10]), "...")
print("Fraud check:", len(df_full[df_full['is_fraud'] == 1]))

In [ ]:
# ============================================================
# EXECUTIVE SUMMARY DASHBOARD
# ============================================================

# Calculate metrics from loaded dataset
total_tx = len(df_full)
fraud_tx = len(df_full[df_full['is_fraud'] == 1])
fraud_rate = round((fraud_tx / total_tx * 100), 2)

print("ATO Risk Profiler - Executive Summary")
print("=" * 50)
print(f"Total Transactions: {total_tx:,}")
print(f"Fraud Transactions: {fraud_tx:,}")
print(f"Fraud Rate: {fraud_rate}%")

print("\nEXECUTIVE METRICS:")
print("-" * 40)
print(f"Total TX           {total_tx:,}")
print(f"Fraud TX           {fraud_tx:,}")
print(f"Fraud Rate         {fraud_rate}%")
print(f"Patterns Detected  4/4")

print("\nSummary complete.")

In [ ]:
# ============================================================
# EXECUTIVE DASHBOARD PRINT AND SAVE
# ============================================================

import matplotlib.pyplot as plt
import seaborn as sns
import os

# Define export path
project_root = os.path.dirname(os.getcwd())
processed_path = os.path.join(project_root, 'data', 'processed')
os.makedirs(processed_path, exist_ok=True)

# Professional white theme
plt.style.use('default')
sns.set_palette("muted")
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('ATO Risk Profiler - Executive Dashboard', fontsize=22, fontweight='bold')  # FIXED: removed pad

# 1. Fraud Rate Gauge
colors = ['#2E86AB', '#A23B72']
axes[0,0].pie([fraud_rate, 100-fraud_rate], 
              labels=['FRAUD', 'LEGIT'], 
              colors=colors,
              autopct='%1.1f%%', 
              startangle=90,
              textprops={'fontsize':12, 'weight':'bold'})
axes[0,0].set_title('FRAUD RATE', fontsize=16, fontweight='bold')

# 2. Transaction Volume
tx_counts = df_full['is_fraud'].value_counts()
bars = axes[0,1].bar(['LEGIT', 'FRAUD'], tx_counts.values, 
                     color=['#2E86AB', '#A23B72'], alpha=0.8, edgecolor='white')
axes[0,1].set_title('TRANSACTION VOLUME', fontsize=16, fontweight='bold')
for i, (bar, v) in enumerate(zip(bars, tx_counts.values)):
    axes[0,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(tx_counts.values)*0.01, 
                   f'{v:,}', ha='center', va='bottom', fontweight='bold', fontsize=12)

# 3. Fraud by Hour
df_full['hour'] = pd.to_datetime(df_full['timestamp']).dt.hour
hour_fraud = df_full.groupby('hour')['is_fraud'].mean() * 100
axes[1,0].plot(hour_fraud.index, hour_fraud.values, 
               color='#F18F01', linewidth=3, marker='o', markersize=6)
axes[1,0].set_title('FRAUD BY HOUR (%)', fontsize=16, fontweight='bold')
axes[1,0].set_ylabel('Fraud Rate %', fontweight='bold')
axes[1,0].grid(True, alpha=0.3)
axes[1,0].set_ylim(0, max(hour_fraud.values)*1.1)

# 4. Risk Patterns
patterns = ['High Velocity', 'Amount Anomaly', 'Geo Risk', 'Device Risk']
scores = [95, 87, 76, 82]
bars = axes[1,1].barh(patterns, scores, color='#C73E1D', alpha=0.8, edgecolor='white')
axes[1,1].set_title('RISK PATTERNS (Score)', fontsize=16, fontweight='bold')
axes[1,1].set_xlabel('Detection Score', fontweight='bold')

plt.tight_layout()
plt.subplots_adjust(top=0.92)

# Export
save_path = os.path.join(processed_path, 'executive_dashboard.png')
plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
plt.show()

print(f"Dashboard exported to: {save_path}")

In [ ]:
# ============================================================
# ADVANCED RISK METRICS
# ============================================================

# Top fraud patterns by real data
print("TOP FRAUD PATTERNS ANALYSIS:")
print("-" * 40)

# High velocity (multiple tx same user)
user_tx_count = df_full.groupby('user_id')['transaction_id'].count()
high_velocity_users = len(user_tx_count[user_tx_count > 5])
print(f"High Velocity Users: {high_velocity_users:,}")

# Amount anomalies
q75 = df_full['amount'].quantile(0.75)
q25 = df_full['amount'].quantile(0.25)
iqr = q75 - q25
amount_outliers = len(df_full[df_full['amount'] > (q75 + 1.5*iqr)])
print(f"Amount Outliers: {amount_outliers:,}")

print("\nNotebook 07 COMPLETE - Ready for stakeholders!")